# Fragment 5

This fragment recovers support for negative forms, lost after introducing tenses.

This fragment is able to interpret the following sentences from the previous fragment:
* "John runs and Mary does not run" as `(run john)∧¬(run mary)`
* "the dog runs" as `run dog`
* "John loves Mary" as `love john mary`
* "it is necessary that John runs" as `□(run john)`
* (broken) "it is not possible that the dog kills John" as `¬◇(kill dog john)`
* (broken) "it is not the case that John loves Mary" as `¬(love john mary)`
* "John has to run" as `□(run john)`
* "the dog is not allowed to kill John" as `¬◇(kill dog john)`
* "John has to be allowed to run" as `□◇(run john)`
* "John will run" as `F(run john)`
* "John ran" as `P(run john)`
* "John had run" as `PP(run john)`
* "John will have run" as `FP(run john)`
* "John would run" as `PF(run john)`
* "John would have run" as `PFP(run john)`
* "John had to run" as `P□(run john)`
* "John had had to run" as `PP□(run john)`
* "John will have to run" as `F□(run john)`
* "John will have had to run" as `FP□(run john)`
* "John will be allowed to run" as `F◇(run john)`
* "John was allowed to run" as `P◇(run john)`

This fragment is able to interpret the following new sentences:
* "John does run" as `run john`
* "John did run" as `P(run john)`
* "John does have to run" as `□(run john)`
* "John will not run" as `¬F(run john)` or `F¬(run john)`
* "John would not run" as `¬PF(run john)`, `PF¬(run john)` or `P¬F(run john)`
* "John did not have to run" as `¬P□(run john)` or `P¬□(run john)`
* (incorrect) "John would not have run" as `¬PFP(run john)`, `P¬FP(run john)`, `PF¬P(run john)` or (wrong) `PFP¬(run john)`

## Parsing

In [1]:
abstract TensesGrammar = {
    cat
        S;     -- Sentence
        SMod;  -- Sentence modifier
        Vi;    -- Intransitive verb
        Vt;    -- Transitive verb
        Vmod;  -- Modal verb
        VP;    -- Verb phrase
        Npr;   -- Proper noun
        N;     -- Common noun
        NP;    -- Noun phrase
        Conj;  -- Conjunction
        Mod;   -- Modal modifier
    fun
        -- Simple verb form sentence: "John runs"
        ssimp : NP -> VP -> S;
        -- Complex verb form sentence: "John does not run"
        scomp : NP -> VP -> S;
        -- Connected sentence:
        -- "John runs and Mary doesn't run"
        sconj : S -> Conj -> S -> S;
        -- Modified sentence: "it is necessary that John runs"
        -- (For be verbs only the complex form exists)
        smod : SMod -> S -> S;
        
        -- Build a verb phrase from an intransitive verb: "run"
        vpvi : Vi -> VP;
        -- Build a verb phrase from a transitive verb and a noun phrase
        -- "love" + "Mary" -> "love Mary"
        vpvt : Vt -> NP -> VP;
        -- Apply the past tense operator
        vppast : VP -> VP;
        -- apply the future tense operator
        vpfut : VP -> VP;
        -- Apply a modal verb to an existing verb phrase
        -- "have to" + "run" -> "have to run"
        vpvmod : Vmod -> VP -> VP;
        -- Apply a modal modifier to an existing verb phrase
        vpmod : Mod -> VP -> VP;
        
        -- Build a noun phrase from a proper noun: "John"
        npnpr : Npr -> NP;
        -- Build a noun phrase from a common noun
        -- "dog" -> "the dog"
        npn : N -> NP;
        
        -- Verbs
        run, jump, laugh, sing, howl, scream : Vi;
        read, poison, eat, like, love, loath, kick, see, understand, kill : Vt;
        haveto, beallowedto : Vmod;
        bepossible, benecessary, bethecase : SMod;
        -- Nouns
        prudence, ethel, chester, jo, bertie, fiona, berta, john, mary, peter : Npr;
        book, cake, katze, golfer, dog, lecturer, student, singer : N;
        -- Conjunctions
        and, or : Conj;
        -- Modal modifiers
        not : Mod;
}

Defined TensesGrammar

In [2]:
concrete TensesGrammarEng of TensesGrammar = {
    param
        Tense = Present | Past | Future;
    oper
        -- The type of the conjugated form of a verb
        VCType : Type = {
            simple : Str;
            prefix : Str;
            suffix : Str
        };
        -- The type of a verb
        VType : Type = {
            base : Str;
            singleTense : Tense => VCType;
            doubleTense : Tense => Tense => VCType;
            tripleTense : Tense => Tense => Tense => VCType
        };
    lincat
        S = Str;        -- Sentence
        SMod = VType;   -- Sentence modifier
        Vi = VType;     -- Intransitive verb
        Vt = VType;     -- Transitive verb
        Vmod = VType;   -- Modal verb
        VP = VType;     -- Verb phrase
        Npr = Str;      -- Proper noun
        N = Str;        -- Common noun
        NP = Str;       -- Noun phrase
        Conj = Str;     -- Conjunction
        Mod = Str;      -- Modal modifier
    oper
        -- None workaround
        None : Str = "[ERROR]";
        
        -- Helper function construct a conjugated form of a verb
        mkVC = overload {
            mkVC : Str -> Str -> Str -> VCType = \simple,pref,suf -> { simple = simple ; prefix = pref ; suffix = suf };
            mkVC : Str -> Str -> VCType = \pref,suf -> { simple = None ; prefix = pref ; suffix = suf };
            mkVC : Str -> VCType = \pref -> { simple = pref ; prefix = pref ; suffix = [] };
        };
        
        -- None workaround for conjugated form of a verb
        VCNone : VCType = { simple = None ; prefix = None ; suffix = None };
        
        -- Helper function to construct a verb
        mkV_ : Str -> Str -> VCType -> VCType -> VType = \vb,vpart,vpres,vpast -> {
            base = vb;
            singleTense = table {
                Present => vpres;
                Past => vpast;
                Future => mkVC "will" vb
            };
            doubleTense = table {
                Present => \\_ => VCNone;
                Past => table {
                    Present => VCNone;
                    Past => mkVC "had" vpart;
                    Future => mkVC "will" ("have" ++ vpart)
                };
                Future => table {
                    Past => mkVC "would" vb;
                    _ => VCNone
                }
            };
            tripleTense = table {
                Present => \\_,_ => VCNone;
                Past => table {
                    Present => \\_ => VCNone;
                    Past => \\_ => VCNone;
                    Future => table {
                        Past => mkVC "would" ("have" ++ vb);
                        _ => VCNone
                    }
                };
                Future => \\_,_ => VCNone
            }
        };
        
        -- Helper function to construct a verb
        mkV = overload {
            mkV : Str -> Str -> Str -> VType =
                \vb,vpast,vpart -> mkV_ vb vpart (mkVC (vb + "s") "does" vb) (mkVC vpast "did" vb);
            mkV : Str -> Str -> Str -> Str -> VType =
                \vb,vpres,vpast,vpart -> mkV_ vb vpart (mkVC vpres "does" vb) (mkVC vpast "did" vb);
        };
        -- Helper function to construct a be verb
        -- mkBeV "possible" = { pos = "is possible" ; neg = "is not possible"}
        mkBeV : Str -> VType = \s -> mkV_ ("be" ++ s) ("been" ++ s) (mkVC "is" s) (mkVC "was" s);
        
        -- Helper function to apply a tense operator to a verb
        applyTense : VType -> Tense -> VType = \v,tense -> {
            base = None;
            singleTense = table {
                Present => v.singleTense ! tense;
                x => v.doubleTense ! tense ! x
            };
            doubleTense = v.tripleTense ! tense;
            tripleTense = \\_,_,_ => VCNone
        };
        
        -- Helper function to append a string to a conjugated form of a verb
        vcconcat : VCType -> Str -> VCType = \vc,s -> {
            simple = vc.simple ++ s;
            prefix = vc.prefix;
            suffix = vc.suffix ++ s;
        };
        
        -- Helper function to append a string to verb
        vpconcat : VType -> Str -> VType = \v,s -> {
            base = v.base ++ s;
            singleTense = \\t => vcconcat (v.singleTense ! t) s;
            doubleTense = \\t1,t2 => vcconcat (v.doubleTense ! t1 ! t2) s;
            tripleTense = \\t1,t2,t3 => vcconcat (v.tripleTense ! t1 ! t2 ! t3) s;
        };
        
        -- Helper function to apply a modifier to a conjugated form of a verb
        vcapplymod : Str -> VCType -> VCType = \mod,vc -> {
            simple = None;
            prefix = vc.prefix;
            suffix = mod ++ vc.suffix;
        };
        
        -- Helper function to apply a modifier to a verb
        vpapplymod : Str -> VType -> VType = \mod,v -> {
            base = mod ++ v.base;
            singleTense = \\t => vcapplymod mod (v.singleTense ! t);
            doubleTense = \\t1,t2 => vcapplymod mod (v.doubleTense ! t1 ! t2);
            tripleTense = \\t1,t2,t3 => vcapplymod mod (v.tripleTense ! t1 ! t2 ! t3);
        };
    lin
        -- Simple verb form sentence: "John runs"
        -- ssimp : NP -> VP -> S
        ssimp npr v = npr ++ (v.singleTense ! Present).simple;
        -- Complex verb form sentence: "John does not run"
        -- scomp : NP -> VP -> S
        scomp npr v = npr ++ (v.singleTense ! Present).prefix ++ (v.singleTense ! Present).suffix;
        -- Connected sentence:
        -- "John runs and Mary doesn't run"
        -- sconj : S -> Conj -> S -> S
        sconj s1 c s2 = s1 ++ c ++ s2;
        -- Modified sentence: "it is necessary that John runs"
        -- (For be verbs only the complex form exists)
        -- smod : SMod -> S -> S
        smod mod s = "it" ++ (mod.singleTense ! Present).prefix ++ (mod.singleTense ! Present).suffix ++ "that" ++ s;
        
        -- Build a verb phrase from an intransitive verb: "run"
        -- vpvi : Vi -> VP
        vpvi v = v;
        -- Build a verb phrase from a transitive verb and a noun phrase
        -- "love" + "Mary" -> "love Mary"
        -- vpvt : Vt -> NP -> VP
        vpvt v np = vpconcat v np;
        -- Apply the past tense operator
        -- vppast : VP -> VP
        vppast vp = applyTense vp Past;
        -- apply the future tense operator
        -- vpfut : VP -> VP
        vpfut vp = applyTense vp Future;
        -- Apply a modal verb to an existing verb phrase
        -- "have to" + "run" -> "have to run"
        -- vpvmod : Vmod -> VP -> VP
        vpvmod vmod vp = vpconcat vmod vp.base;
        -- Apply a modal modifier to an existing verb phrase
        -- vpmod : Mod -> VP -> VP
        vpmod mod vp = vpapplymod mod vp;
        
        -- Build a noun phrase from a proper noun: "John"
        -- npnpr : Npr -> NP
        npnpr npr = npr;
        -- Build a noun phrase from a common noun
        -- "dog" -> "the dog"
        -- npn : N -> NP
        npn n = "the" ++ n;
        
        -- Intrasitive verbs
        run = mkV "run" "ran" "run"; jump = mkV "jump" "jumped" "jumped"; laugh = mkV "laugh" "laughed" "laughed";
        sing = mkV "sing" "sang" "sung"; howl = mkV "howl" "howled" "howled"; scream = mkV "scream" "screamed" "screamed";
        -- Transitive verbs
        read = mkV "read" "read" "read"; poison = mkV "poison" "poisoned" "poisoned"; eat = mkV "eat" "ate" "eaten";
        like = mkV "like" "liked" "liked"; love = mkV "love" "loved" "loved"; loath = mkV "loath" "loathed" "loathed";
        kick = mkV "kick" "kicked" "kicked"; see = mkV "see" "saw" "seen"; understand = mkV "understand" "understood" "understood";
        kill = mkV "kill" "killed" "killed";
        -- Modal verbs
        haveto = mkV "have to" "has to" "had to" "had to"; beallowedto = mkBeV "allowed to";
        bepossible = mkBeV "possible"; benecessary = mkBeV "necessary"; bethecase = mkBeV "the case";
        -- Nouns
        prudence = "Prudence"; ethel = "Ethel"; chester = "Chester"; jo = "Jo"; bertie = "Bertie"; fiona = "Fiona"; berta = "Berta";
        john = "John"; mary = "Mary"; peter = "Peter";
        book = "book"; cake = "cake"; katze = "cat"; golfer = "golfer"; dog = "dog"; lecturer = "lecturer"; student = "student";
        singer = "singer"; 
        -- Conjunctions
        and = "and"; or = "or";
        -- Modal modifiers
        not = "not";
}

### Parsing test cases (old)

In [3]:
parse "John runs"

ssimp (npnpr john) (vpvi run)

In [4]:
parse "John does not run"

scomp (npnpr john) (vpmod not (vpvi run))

In [5]:
parse "John runs and Mary does not run"

sconj (ssimp (npnpr john) (vpvi run)) and (scomp (npnpr mary) (vpmod not (vpvi run)))

In [6]:
parse "John does not jump or Mary runs"

sconj (scomp (npnpr john) (vpmod not (vpvi jump))) or (ssimp (npnpr mary) (vpvi run))

In [7]:
parse "the dog runs"

ssimp (npn dog) (vpvi run)

In [8]:
parse "John loves Mary"

ssimp (npnpr john) (vpvt love (npnpr mary))

In [9]:
parse "John loves Mary and Mary does not love John"

sconj (ssimp (npnpr john) (vpvt love (npnpr mary))) and (scomp (npnpr mary) (vpmod not (vpvt love (npnpr john))))

In [10]:
parse "it is necessary that John runs"

smod benecessary (ssimp (npnpr john) (vpvi run))

In [11]:
parse "it is not possible that the dog kills John"

The parser failed at token 3: "not"

In [12]:
parse "it is not the case that John loves Mary"

The parser failed at token 3: "not"

In [13]:
parse "John has to run"

ssimp (npnpr john) (vpvmod haveto (vpvi run))

In [14]:
parse "the dog is not allowed to kill John"

scomp (npn dog) (vpmod not (vpvmod beallowedto (vpvt kill (npnpr john))))

In [15]:
parse "John has to be allowed to run"

ssimp (npnpr john) (vpvmod haveto (vpvmod beallowedto (vpvi run)))

In [16]:
parse "John will run"

scomp (npnpr john) (vpfut (vpvi run))

In [17]:
parse "John ran"

ssimp (npnpr john) (vppast (vpvi run))

In [18]:
parse "John had run"

scomp (npnpr john) (vppast (vppast (vpvi run)))

In [19]:
parse "John will have run"

scomp (npnpr john) (vpfut (vppast (vpvi run)))

In [20]:
parse "John would run"

scomp (npnpr john) (vppast (vpfut (vpvi run)))

In [21]:
parse "John would have run"

scomp (npnpr john) (vppast (vpfut (vppast (vpvi run))))

In [22]:
parse "John had to run"

ssimp (npnpr john) (vppast (vpvmod haveto (vpvi run)))

In [23]:
parse "John had had to run"

scomp (npnpr john) (vppast (vppast (vpvmod haveto (vpvi run))))

In [24]:
parse "John will have to run"

scomp (npnpr john) (vpfut (vpvmod haveto (vpvi run)))

In [25]:
parse "John will have had to run"

scomp (npnpr john) (vpfut (vppast (vpvmod haveto (vpvi run))))

In [26]:
parse "John will be allowed to run"

scomp (npnpr john) (vpfut (vpvmod beallowedto (vpvi run)))

In [27]:
parse "John was allowed to run"

scomp (npnpr john) (vppast (vpvmod beallowedto (vpvi run)))

### Parsing test cases (new)

In [28]:
parse "John does run"

scomp (npnpr john) (vpvi run)

In [29]:
parse "John did run"

scomp (npnpr john) (vppast (vpvi run))

In [30]:
parse "John does have to run"

scomp (npnpr john) (vpvmod haveto (vpvi run))

In [31]:
parse "John will not run"

scomp (npnpr john) (vpfut (vpmod not (vpvi run)))

scomp (npnpr john) (vpmod not (vpfut (vpvi run)))

In [32]:
parse "John would not run"

scomp (npnpr john) (vpmod not (vppast (vpfut (vpvi run))))

scomp (npnpr john) (vppast (vpfut (vpmod not (vpvi run))))

scomp (npnpr john) (vppast (vpmod not (vpfut (vpvi run))))

In [33]:
parse "John did not have to run"

scomp (npnpr john) (vpmod not (vppast (vpvmod haveto (vpvi run))))

scomp (npnpr john) (vppast (vpmod not (vpvmod haveto (vpvi run))))

In [34]:
parse "John would not have run"

scomp (npnpr john) (vpmod not (vppast (vpfut (vppast (vpvi run)))))

scomp (npnpr john) (vppast (vpfut (vpmod not (vppast (vpvi run)))))

scomp (npnpr john) (vppast (vpfut (vppast (vpmod not (vpvi run)))))

scomp (npnpr john) (vppast (vpmod not (vpfut (vppast (vpvi run)))))

In [35]:
parse "the dog had not killed John"

scomp (npn dog) (vpmod not (vppast (vppast (vpvt kill (npnpr john)))))

scomp (npn dog) (vppast (vpmod not (vppast (vpvt kill (npnpr john)))))

scomp (npn dog) (vppast (vppast (vpmod not (vpvt kill (npnpr john)))))

## Semantics construction

In [36]:
theory plnq : ur:?LF =
    proposition : type ❘ # o ❙
    individual : type ❘ # ι ❙
    
    not : o ⟶ o         ❘ # ¬ 1 prec 100 ❙
    and : o ⟶ o ⟶ o     ❘ # 1 ∧ 2 prec 90 ❙
    
    or : o ⟶ o ⟶ o      ❘ # 1 ∨ 2 prec 80
                        ❘ = [a,b] ¬ (¬ a ∧ ¬ b) ❙
❚

theory modalplnq : ur:?LF =
    include ?plnq ❙
    box : o ⟶ o     ❘ # □ 1 prec 100 ❙
    diamond : o ⟶ o ❘ # ◇ 1 prec 100 ❙
❚

theory tenseplnq : ur:?LF =
    include ?modalplnq ❙
    
    // FΦ: it will at some stage in the future be the case that Φ ❙
    futF : o ⟶ o ❘ # F 1 prec 100 ❙
    // PΦ: it was at some stage in the past the case that Φ ❙
    pastP : o ⟶ o ❘ # P 1 prec 100 ❙
❚

theory plnqTest : ?tenseplnq =
    // individuals ❙
    prudence : ι ❙ ethel : ι ❙ chester : ι ❙ jo : ι ❙ bertie : ι ❙ fiona : ι ❙ berta : ι ❙ john : ι ❙ mary : ι ❙ peter : ι ❙
    book : ι ❙ cake : ι ❙ cat : ι ❙ golfer : ι ❙ dog : ι ❙ lecturer : ι ❙ student : ι ❙ singer : ι ❙
    // verbs ❙
    run : ι ⟶ o ❙ jump : ι ⟶ o ❙ laugh : ι ⟶ o ❙ sing : ι ⟶ o ❙ howl : ι ⟶ o ❙ scream : ι ⟶ o ❙
    read : ι ⟶ ι ⟶ o ❙ poison : ι ⟶ ι ⟶ o ❙ eat : ι ⟶ ι ⟶ o ❙ like : ι ⟶ ι ⟶ o ❙ love : ι ⟶ ι ⟶ o ❙ loath : ι ⟶ ι ⟶ o ❙
    kick : ι ⟶ ι ⟶ o ❙ see : ι ⟶ ι ⟶ o ❙ understand : ι ⟶ ι ⟶ o ❙ kill : ι ⟶ ι ⟶ o ❙
❚

Created theory plnq

In [37]:
view semantics : http://mathhub.info/comma/jupyter/TensesGrammar.gf?TensesGrammar -> ?plnqTest =
    S = o ❙             // Sentence ❙
    SMod = o ⟶ o ❙      // Sentence modifier ❙
    Vi = ι ⟶ o ❙        // Intransitive verb ❙
    Vt = ι ⟶ ι ⟶ o ❙    // Transitive verb ❙
    Vmod = o ⟶ o ❙      // Modal verb ❙
    VP = ι ⟶ o ❙        // Verb phrase ❙
    Npr = ι ❙           // Proper noun ❙
    N = ι ❙             // Common noun, as a simplification modeled incorrectly
                        // as an individual ❙
    NP = ι ❙            // Noun phrase ❙
    Conj = o ⟶ o ⟶ o ❙  // Conjunction ❙
    Mod = o ⟶ o ❙       // Modal modifier ❙
    
    // Simple verb form sentence: "John runs"
    // ssimp : NP -> VP -> S ❙
    ssimp = [npr, v] v npr ❙
    // Complex verb form sentence: "John does not run"
    // scomp : NP -> VP -> S ❙
    scomp = [npr, v] v npr ❙
    // Connected sentence:
    // "John runs and Mary doesn't run"
    // sconj : S -> Conj -> S -> S ❙
    sconj = [s1, conj, s2] conj s1 s2 ❙
    // Modified sentence: "it is necessary that John runs"
    // smod : SMod -> S -> S ❙
    smod = [mod, s] mod s ❙
    
    // Build a verb phrase from an intransitive verb: "run"
    // vpvi : Vi -> VP ❙
    vpvi = [v] v ❙
    // Build a verb phrase from a transitive verb and a noun phrase
    // "love" + "Mary" -> "love Mary"
    // vpvt : Vt -> NP -> VP ❙
    vpvt = [vt, addressee] [actor] vt actor addressee ❙
    // Apply the past tense operator
    // vppast : VP -> VP ❙
    vppast = [v] [i] P (v i) ❙
    // apply the future tense operator
    // vpfut : VP -> VP ❙
    vpfut = [v] [i] F (v i) ❙
    // Apply a modal verb to an existing verb phrase
    // "have to" + "run" -> "have to run"
    // vpvmod : Vmod -> VP -> VP ❙
    vpvmod = [vmod, vp] [i] vmod (vp i) ❙
    // Apply a modal modifier to an existing verb phrase
    // vpmod : Mod -> VP -> VP ❙
    vpmod = [mod, vp] [i] mod (vp i) ❙
        
    // Build a noun phrase from a proper noun: "John"
    // npnpr : Npr -> NP ❙
    npnpr = [npr] npr ❙
    // Build a noun phrase from a common noun
    // "dog" -> "the dog"
    // npn : N -> NP ❙
    npn = [n] n ❙
    
    // Verbs ❙
    run = run ❙ jump = jump ❙ laugh = laugh ❙ sing = sing ❙ howl = howl ❙ scream = scream ❙
    read = read ❙ poison = poison ❙ eat = eat ❙ like = like ❙ love = love ❙ loath = loath ❙ kick = kick ❙ see = see ❙ understand = understand ❙ kill = kill ❙
    haveto = box ❙ beallowedto = diamond ❙
    bepossible = diamond ❙ benecessary = box ❙ bethecase = [p] p ❙
    // Nouns ❙
    prudence = prudence ❙ ethel = ethel ❙ chester = chester ❙ jo = jo ❙ bertie = bertie ❙ fiona = fiona ❙ berta = berta ❙ john = john ❙ mary = mary ❙ peter = peter ❙
    book = book ❙ cake = cake ❙ katze = cat ❙ golfer = golfer ❙ dog = dog ❙ lecturer = lecturer ❙ student = student ❙ singer = singer ❙
    // Conjunctions ❙
    and = and ❙ or = or ❙
    // Modal modifiers ❙
    not = not ❙
❚

Created view semantics

### Semantics construction test cases (old)

In [38]:
parse "John runs" | construct

run john

In [39]:
parse "John does not run" | construct

¬(run john)

In [40]:
parse "John runs and Mary does not run" | construct

(run john)∧¬(run mary)

In [41]:
parse "John does not jump or Mary runs" | construct

¬(jump john)∨(run mary)

In [42]:
parse "the dog runs" | construct

run dog

In [43]:
parse "John loves Mary" | construct

love john mary

In [44]:
parse "John loves Mary and Mary does not love John" | construct

(love john mary)∧¬(love mary john)

In [45]:
parse "it is necessary that John runs" | construct

□(run john)

In [46]:
parse "John has to run" | construct

□(run john)

In [47]:
parse "the dog is not allowed to kill John" | construct

¬◇(kill dog john)

In [48]:
parse "John has to be allowed to run" | construct

□◇(run john)

In [49]:
parse "John will run" | construct

F (run john)

In [50]:
parse "John ran" | construct

P (run john)

In [51]:
parse "John had run" | construct

P P (run john)

In [52]:
parse "John will have run" | construct

F P (run john)

In [53]:
parse "John would run" | construct

P F (run john)

In [54]:
parse "John would have run" | construct

P F P (run john)

In [55]:
parse "John had to run" | construct

P □(run john)

In [56]:
parse "John had had to run" | construct

P P □(run john)

In [57]:
parse "John will have to run" | construct

F □(run john)

In [58]:
parse "John will have had to run" | construct

F P □(run john)

In [59]:
parse "John will be allowed to run" | construct

F ◇(run john)

In [60]:
parse "John was allowed to run" | construct

P ◇(run john)

### Semantics construction test cases (new)

In [61]:
parse "John does run" | construct

run john

In [62]:
parse "John did run" | construct

P (run john)

In [63]:
parse "John does have to run" | construct

□(run john)

In [64]:
parse "John will not run" | construct

F ¬(run john)

¬F (run john)

In [65]:
parse "John would not run" | construct

¬P F (run john)

P F ¬(run john)

P ¬F (run john)

In [66]:
parse "John did not have to run" | construct

¬P □(run john)

P ¬□(run john)

In [67]:
parse "John would not have run" | construct

¬P F P (run john)

P F ¬P (run john)

P F P ¬(run john)

P ¬F P (run john)

In [68]:
parse "the dog had not killed John" | construct

¬P P (kill dog john)

P ¬P (kill dog john)

P P ¬(kill dog john)